<a href="https://colab.research.google.com/github/felipemanfrin/NLP/blob/master/QAbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np

In [2]:
with open ('train_qa.txt', 'rb') as f:
  train_data = pickle.load(f)

In [3]:
with open ('test_qa.txt', 'rb') as f:
  test_data = pickle.load(f)

In [4]:
type(test_data)

list

In [5]:
type(train_data)

list

In [6]:
len(train_data)

10000

In [7]:
len(test_data)

1000

In [9]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [19]:
' '.join(train_data[0][0])

['Mary',
 'moved',
 'to',
 'the',
 'bathroom',
 '.',
 'Sandra',
 'journeyed',
 'to',
 'the',
 'bedroom',
 '.']

In [12]:
' '.join(train_data[0][1])


'Is Sandra in the hallway ?'

In [14]:
(train_data[0][2])

'no'

In [15]:
all_data = test_data + train_data

In [16]:
len(all_data)

11000

In [18]:
set(train_data[0][0])

{'.',
 'Mary',
 'Sandra',
 'bathroom',
 'bedroom',
 'journeyed',
 'moved',
 'the',
 'to'}

In [20]:
vocab = set()

for story, question, anser in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))

In [21]:
vocab.add('no')

In [22]:
vocab.add('yes')

In [23]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [24]:
vocab_len = len(vocab) + 1

In [25]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [26]:
vocab_len

38

In [27]:
# LONGEST STORY
all_story_lens = [len(data[0]) for data in all_data]

In [31]:
#all_story_lens
max_story_len = max(all_story_lens)
max_question_len = max([len(data[1]) for data in all_data])

In [32]:
max_question_len

6

In [44]:
from keras.preprocessing.sequence import  pad_sequences
from keras.preprocessing.text import Tokenizer

In [46]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [47]:
tokenizer.word_index

{'.': 32,
 '?': 12,
 'apple': 2,
 'back': 16,
 'bathroom': 10,
 'bedroom': 28,
 'daniel': 18,
 'discarded': 36,
 'down': 3,
 'dropped': 24,
 'football': 33,
 'garden': 15,
 'got': 5,
 'grabbed': 11,
 'hallway': 26,
 'in': 34,
 'is': 8,
 'john': 23,
 'journeyed': 6,
 'kitchen': 22,
 'left': 19,
 'mary': 1,
 'milk': 9,
 'moved': 14,
 'no': 20,
 'office': 31,
 'picked': 7,
 'put': 25,
 'sandra': 27,
 'the': 30,
 'there': 21,
 'to': 4,
 'took': 37,
 'travelled': 29,
 'up': 13,
 'went': 17,
 'yes': 35}

In [48]:
train_story_text = []
train_question_text = []
train_answers = []

In [50]:
for story, question, answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)
  train_answers.append(train_answers)

In [56]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [57]:
len(train_story_seq)

10001

In [ ]:
train_story_text

In [ ]:
train_story_seq

In [90]:
def vectorize_stories (data, word_index=tokenizer.word_index, max_story_len=max_story_len, max_question_len=max_question_len):
    # X = STORIES
    X = []
    # Xq = QUERY/QUESTION
    Xq = []
    # Y = CORRECT ANSWER
    Y = []
    
    
    for story, query, answer in data:
        
        # Grab the word index for every word in story
        x = [word_index[word.lower()] for word in story]
        # Grab the word index for every word in query
        xq = [word_index[word.lower()] for word in query]
        
        # Grab the Answers (either Yes/No so we don't need to use list comprehension here)
        # Index 0 is reserved so we're going to use + 1
        y = np.zeros(len(word_index) + 1)
        
        # Now that y is all zeros and we know its just Yes/No , we can use numpy logic to create this assignment
        #
        y[word_index[answer]] = 1
        
        # Append each set of story,query, and answer to their respective holding lists
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    # Finally, pad the sequences based on their max length so the RNN can be trained on uniformly long sequences.
        
    # RETURN TUPLE FOR UNPACKING
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

In [91]:
inputs_train, queries_train, answer_train = vectorize_stories(train_data)

In [92]:
inputs_test, queries_test, answer_test = vectorize_stories(train_data)

In [93]:
tokenizer.word_index['yes']

35

In [94]:
tokenizer.word_index['no']

20

In [95]:
sum(answer_test)

array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0., 4988.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0., 5012.,
          0.,    0.])